In [43]:
import numpy as np
import pandas as pd
import requests
import os
from nltk import word_tokenize
from nltk.tokenize import sent_tokenize
import string
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
from rake_nltk import Rake
import gensim
from gensim import corpora

In [65]:
def load_file(address):
    if os.path.exists(address)==False:
        print('path not found')
    if address.endswith('.txt'):
        f=open(address)
        g=f.read()
        return g
    else:
        print('Please give a txt file')

In [59]:
def save_it(text,filename,address=os.getcwd()):
    if os.path.exists(address)==False:
        print('Path Not Found')
    else:
        with open(address+"\\"+filename+".txt", 'w') as f:
            f.write(text)

In [60]:
def load_url(link_provided):
    ua = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    from requests.exceptions import ConnectionError
    try:
        response = requests.get(link_provided, headers={"User-Agent": ua})
    except ConnectionError:
        print ('Failed to open url.')
        return
    if response.status_code!=200:
        print("Bad response code :", response.status_code, link_provided)
        return
    if response.status_code == 200:
        f=response.content
    from bs4 import  BeautifulSoup
    text=BeautifulSoup(f,"html.parser")
    text=text.getText()
    text=text.strip()
    text=text.rstrip('\n')
    text="\n".join(item for item in text.split('\n') if item)
    text=text.replace('\n','.')
    text=text.rstrip('\n')
    text = text.encode('utf-8').decode('ascii', 'ignore')
    base=input("Give The Address where you want to save folder(Enter No if you want it to save in current directory, Enter yes if you don't want to save)\n ")
    if(str.lower(base)=="no"):
        filename=input('Enter Filename:- \nCaution:- It will change the current file, if exist, of the same name:\n')
        save_it(text,filename)
    elif str.lower(base)=="yes":
        return text
    else:
        filename=input('Enter Filename: \n Caution:- It will change the current file, if exist, of the same name:\n')
        save_it(text,filename,base)

In [47]:
def check_for_words_after_link(text,words):
    words=str.lower(words)
    for i in range(0,len(words)):
        if words[i] in text:
            print('Forbidden Word Found is ', words[i])
            return
    print("Forbidden words not found")

In [48]:
def check_for_words_before_link(link_provided,words):
    link_provided=str.lower(link_provided)
    for i in range(0,len(words)):
        if words[i] in link_provided:
            print('Contain Forbidden Words\n')
        else:
            load_url(link_provided)

In [49]:
punctuation=list(string.punctuation)

In [50]:
def preprocess(text):
    text=word_tokenize(text)
    answer=[i for i  in text if i not in punctuation]
    for i in range(0,len(answer)):
        answer[i]=str.lower(answer[i])
    stopwords = nltk.corpus.stopwords.words('english')
    newanswer=[]
    for i in range(0,len(answer)):
        if answer[i] not in stopwords:
            newanswer.append(answer[i])
    porter_stemmer = PorterStemmer()
    stem_text=[]
    for i in range(0,len(newanswer)):
        stem_text.append(porter_stemmer.stem(newanswer[i]))
    wordnet_lemmatizer = WordNetLemmatizer()
    lemm_text=[]
    for i in range(0,len(stem_text)):
        lemm_text.append(wordnet_lemmatizer.lemmatize(stem_text[i]))
    return lemm_text

In [51]:
def cloud_it(words):
    if isinstance(words,list):
        text=""
        for i in range(0,len(words)):
            text=text+words[i]
    else:
        text=words
    wordclou = WordCloud(width = 800, height = 800,background_color ='white',
                min_font_size = 10).generate(text)
    plt.figure(figsize = (8, 8), facecolor = None)
    plt.imshow(wordclou)
    plt.axis("off")
    plt.tight_layout(pad = 0)
    plt.show()

In [52]:
def extract_it(words,n):
    if isinstance(words,list):
        text=""
        for i in range(0,len(words)):
            text=text+words[i]
    else:
        text=words
    r=Rake()
    r.extract_keywords_from_text(text)
    print(r.get_ranked_phrases()[0:n])